In [37]:
from pyspark.sql import SparkSession
from datetime import datetime

In [38]:
spark_server = "spark://spark-master-otmzsp:7077"
minio_server = "http://minio-otmzsp:9000" 
data_inicio_processamento = "2024-09-23"

In [39]:
# Configurando a sessão do Spark com as dependências e variáveis do S3A
# .master(spark_server) \
# .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
spark = SparkSession.builder \
    .appName("Processa Posições Raw") \
    .master(spark_server) \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.endpoint", minio_server) \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [40]:
spark

In [41]:
raw = 's3a://raw/posicoes'
trusted = 's3a://trusted/posicoes'
checkpoint = 's3a://trusted/posicoes_checkpoint'

In [42]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType, TimestampType, FloatType

# Defining the schema for the JSON
schema = StructType([StructField("hr", StringType(), True),
                     StructField("l", ArrayType(StructType([StructField("c", StringType(), True),
                                                            StructField("cl", IntegerType(), True),
                                                            StructField("sl", IntegerType(), True),
                                                            StructField("lt0", StringType(), True),
                                                            StructField("lt1", StringType(), True),
                                                            StructField("qv", IntegerType(), True),
                                                            StructField("vs", ArrayType(StructType([StructField("p", IntegerType(), True),
                                                                                                    StructField("a", BooleanType(), True),
                                                                                                    StructField("ta", TimestampType(), True),
                                                                                                    StructField("py", FloatType(), True),
                                                                                                    StructField("px", FloatType(), True),
                                                                                                    StructField("sv", StringType(), True),
                                                                                                    StructField("is", StringType(), True)
                                                                                                    ])
                                                                                        )
                                                                        )
                                                            ])
                                                )
                                )
                    ])


In [43]:
from pyspark.sql.functions import concat_ws, sha1
from pyspark.sql.functions import (
    explode, col, to_timestamp, when, from_utc_timestamp, to_date
)
from pyspark.sql.types import DoubleType

def transform_data(df):
    # Assumindo que 'df' é o DataFrame original carregado do JSON da SPTrans
    df_final = df.select(
        to_timestamp(col("hr")).alias("veiculo_horario_referencia"),
        explode("l").alias("linha")
    ).select(
        col("veiculo_horario_referencia"),
        col("linha.c").alias("veiculo_letreiro_completo"),
        col("linha.cl").alias("veiculo_linha_codigo"),
        col("linha.sl").alias("veiculo_sentido"),
        col("linha.lt0").alias("veiculo_letreiro_destino"),
        col("linha.lt1").alias("veiculo_letreiro_origem"),
        col("linha.qv").alias("qtde_veiculos_linha"),
        explode("linha.vs").alias("veiculo")
    ).select(
        "*",
        col("veiculo.p").alias("veiculo_prefixo"),
        col("veiculo.a").alias("veiculo_acessibilidade"),
        col("veiculo.ta").alias("veiculo_horario_utc_captura"),
        col("veiculo.py").cast(DoubleType()).alias("veiculo_latitude"),
        col("veiculo.px").cast(DoubleType()).alias("veiculo_longitude")
    ).drop("linha", "veiculo")

    # Tratamento do sentido do veículo
    df_final = df_final.withColumn(
        "veiculo_sentido",
        when(col("veiculo_sentido") == 1, "Bairro")
        .when(col("veiculo_sentido") == 2, "Centro")
        .otherwise(col("veiculo_sentido"))
    )

    # Tratamento da acessibilidade
    df_final = df_final.withColumn(
        "veiculo_acessibilidade",
        when(col("veiculo_acessibilidade") == "true", "Acessível")
        .when(col("veiculo_acessibilidade") == "false", "Não acessível")
        .otherwise("Informação não disponível")
    )

    # Converter horário UTC para horário local (São Paulo, UTC-3)
    df_final = df_final.withColumn(
        "veiculo_horario_local_captura",
        from_utc_timestamp(to_timestamp(col("veiculo_horario_utc_captura")), "America/Sao_Paulo")
    )

    # Adicionar coluna com o tipo de operação da linha
    df_final = df_final.withColumn(
        "tipo_operacao_linha",
        when(col("veiculo_letreiro_completo").substr(-2, 2) == "10", "Linha Base")
        .when(col("veiculo_letreiro_completo").substr(-2, 2).isin("21", "23", "32", "41"), "Linha de Atendimento")
        .otherwise("Outro tipo de operação")
    )

    # Criar uma coluna 'id' com um identificador único usando SHA-256
    df_final = df_final.withColumn(
        "id",
        sha1(
            concat_ws(
                "-", 
                col("veiculo_letreiro_completo"),
                col("veiculo_linha_codigo"),
                col("veiculo_prefixo"),
                col("veiculo_horario_local_captura"),
                col("veiculo_latitude"),
                col("veiculo_longitude")
            )
        )
    )
        # Obtém o timestamp atual
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Imprime o timestamp com a mensagem
    print(f"{timestamp}: define transformation")

    return df_final

In [44]:
from pyspark.sql import functions as F
import time

def send_to_trusted(df_final):
    # Supondo que df_final já está definido e contém uma coluna 'id' e 'veiculo_horario_referencia'

    # Remover registros duplicados com base no campo 'id'
    df_final_unique = df_final.dropDuplicates(["id"])

    # Adiciona uma coluna com o formato 'yyyy-MM-dd-HH' baseado em veiculo_horario_referencia
    df_final_unique = df_final_unique.withColumn(
        "datepartition",
        F.date_format(F.col("veiculo_horario_local_captura"), "yyyy-MM-dd-HH")
    )

    # Ajuste o número de partições, se necessário
    #df_final_unique = df_final_unique.repartition(4, "datepartition")

    # Escreve o stream em CSV particionado por 'yyyy-MM-dd-HH', onde novos dados serão continuamente adicionados
    df_final_unique.write \
        .format("csv") \
        .option("path", trusted) \
        .option("checkpointLocation", checkpoint) \
        .partitionBy("datepartition") \
        .option("header", "true") \
        .mode("append") \
        .save()
    from datetime import datetime
    
    # Obtém o timestamp atual
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Imprime o timestamp com a mensagem
    print(f"{timestamp}: escreveu os dados na trusted")

In [45]:
def load_raw():
    # Leitura incremental de arquivos JSON da camada raw
    df_read = spark.read.format("json").schema(schema).load(raw + f'/datepartition={data_inicio_processamento}')
   
    # Obtém o timestamp atual
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Imprime o timestamp com a mensagem
    print(f"{timestamp}: print carregou os dados da raw")

    #df_read.show()
    return df_read

In [46]:
try:
    import schedule
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "schedule"])
    import schedule


In [47]:
import schedule
import time

def job():
    send_to_trusted(transform_data(load_raw()))

# Executa a função pela primeira vez imediatamente
job()

schedule.every(5).minutes.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)


2024-09-24 21:41:52: print carregou os dados da raw
2024-09-24 21:41:52: define transformation
2024-09-24 21:46:35: escreveu os dados na trusted
2024-09-24 21:46:35: print carregou os dados da raw
2024-09-24 21:46:35: define transformation
2024-09-24 21:51:14: escreveu os dados na trusted
2024-09-24 21:51:14: print carregou os dados da raw
2024-09-24 21:51:14: define transformation
2024-09-24 21:56:08: escreveu os dados na trusted
2024-09-24 21:56:09: print carregou os dados da raw
2024-09-24 21:56:09: define transformation
2024-09-24 22:00:50: escreveu os dados na trusted
2024-09-24 22:00:51: print carregou os dados da raw
2024-09-24 22:00:51: define transformation


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [48]:
spark.stop()